import & setup cơ bản

In [25]:
import pandas as pd
pd.set_option('display.max_columns',500,'float_format', '{:,.2f}'.format) #hien het cac dong, format dau phay, 2 so 0
pd.set_option('display.max_rows', 500)

ĐỌC BÁO CÁO TỪ FILE CLEAN

In [26]:
path_bctc_clean_quarterly = '../../data/clean/vietstock_data/bctc_hsx_hnx_upcom/quarterly/'
file_BCTC = path_bctc_clean_quarterly + 'BCTC_quarterly.xlsx'
file_BCTC_none_financials_realestate_airline = path_bctc_clean_quarterly + "BCTC_none_financials_realestate_airline_quarterly.xlsx"
BCTC = pd.read_excel(file_BCTC,sheet_name=0, header=0)
#BCTC_none_financials_realestate_airline = pd.read_excel(file_BCTC_none_financials_realestate_airline,sheet_name=0, header=0)

Trả lời câu hỏi diễn biến lợi nhuận của toàn thị trường

In [27]:
BCTC['EBITDA'] = BCTC['15. Profit before tax (50=30+40+41)'] + BCTC['Depreciation of fixed assets and properties investment'] -  BCTC['7. Financial expenses'] # thêm EBITDA
BCTC['Market_cap'] = BCTC["1. Owner's capital "]*BCTC['Close Price End of period']/10000
BCTC['Gross_profit_margin'] = BCTC['5. Gross profit (20 = 10 - 11)']*100/(BCTC["3. Net revenue  (10 = 01 - 03)"])
BCTC['Oprating_profit_margin'] = BCTC['11. Operating profit {30 = 20 + (21 - 22) - (24 + 25)}']*100/(BCTC["3. Net revenue  (10 = 01 - 03)"])
BCTC['net_profit_margin'] = BCTC['18. Net profit after tax  (60 = 50 - 51 - 52)']*100/(BCTC["3. Net revenue  (10 = 01 - 03)"])
BCTC['EPS_quarter'] = BCTC['18. Net profit after tax  (60 = 50 - 51 - 52)']/(BCTC["1. Owner's capital "]/10000)
BCTC['Rolling_EPS_4Q'] = BCTC.groupby('VS-Sector L4')['EPS_quarter'].rolling(4).sum().reset_index(level=0, drop=True)
BCTC['PE_quarter'] = BCTC['Market_cap']/(BCTC['18. Net profit after tax  (60 = 50 - 51 - 52)'])
BCTC['Rolling_Profit_4Q'] = BCTC.groupby('VS-Sector L4')['18. Net profit after tax  (60 = 50 - 51 - 52)'].rolling(4).sum().reset_index(level=0, drop=True)
BCTC['Last_Year_Rolling_Profit_4Q'] = BCTC.groupby(['VS-Sector L4'])['Rolling_Profit_4Q'].shift(4)  #lay loi nhuan rolling  cung ky nam truoc
BCTC['YoY_Growth_Rolling_Profit_4Q'] = (BCTC['Rolling_Profit_4Q']/BCTC['Last_Year_Rolling_Profit_4Q']-1)*100
BCTC['ROE(%)'] = BCTC['Rolling_Profit_4Q']*100/BCTC["1. Owner's capital "]
BCTC['ROA(%)'] = BCTC['Rolling_Profit_4Q']*100/BCTC["TOTAL ASSETS (280=100+200)"]
BCTC['Rolling_PE_4Q'] = BCTC['Market_cap']/BCTC['Rolling_Profit_4Q']




sector_exclude = ['Real Estate Development','Real Estate Holding & Development','Scheduled Air Transportation','Depository Credit Intermediation']

Sector_level1_exclude = ['Financials','Real Estate']
Sector_level4_exclude = ['Passenger Airlines ']

BCTC_none_financials_realestate = BCTC[~BCTC['VS-Sector L1'].isin(Sector_level1_exclude)] 
BCTC_none_financials_realestate_airline = BCTC_none_financials_realestate[~BCTC_none_financials_realestate['VS-Sector L4'].isin(Sector_level4_exclude)] 



In [28]:
BCTC.groupby(['Year_Q'])[["TOTAL ASSETS (280=100+200)","B. OWNER'S EQUITY (400=410+430)","1. Owner's capital ",'15. Profit before tax (50=30+40+41)','18. Net profit after tax  (60 = 50 - 51 - 52)','EBITDA']].sum()


,TOTAL ASSETS (280=100+200),B. OWNER'S EQUITY (400=410+430),1. Owner's capital,15. Profit before tax (50=30+40+41),18. Net profit after tax (60 = 50 - 51 - 52),EBITDA
Year_Q,,,,,,
2022Q1,"17,031,477,863,196,608.00","3,291,871,813,564,999.00","1,825,124,966,378,724.00","167,262,839,443,628.00","137,226,872,075,364.00","99,473,536,831,288.00"
2022Q2,"17,731,469,166,798,362.00","3,383,672,388,204,010.00","1,877,735,420,774,621.00","161,068,144,538,761.00","131,066,013,242,561.00","97,188,894,068,585.00"
2022Q3,"18,113,531,705,129,560.00","3,479,473,906,160,057.00","1,925,001,723,216,154.00","147,127,017,024,150.00","113,163,186,994,761.00","83,434,325,657,444.00"
2022Q4,"18,738,564,748,796,332.00","3,536,152,317,779,585.00","1,986,116,215,655,189.00","111,613,984,695,822.00","82,437,492,201,068.00","59,173,646,124,403.00"
2023Q1,"19,053,721,647,011,264.00","3,622,428,715,297,055.00","1,984,567,307,307,387.00","140,243,944,355,003.00","108,922,405,390,016.00","73,629,490,333,577.00"
2023Q2,"19,384,641,583,005,364.00","3,677,106,829,211,450.00","2,009,076,065,073,154.00","144,437,939,945,698.00","112,882,152,640,934.00","81,569,098,424,649.00"
2023Q3,"19,790,989,523,580,272.00","3,772,124,637,010,962.00","2,070,503,788,663,954.00","138,593,566,886,196.00","107,041,136,654,780.00","65,859,380,759,435.00"
2023Q4,"21,030,537,527,000,124.00","3,880,740,434,052,606.00","2,129,959,464,807,399.00","141,766,043,990,823.00","112,748,459,352,408.00","75,290,296,602,996.00"
2024Q1,"21,174,986,011,275,804.00","3,994,688,084,942,986.00","2,129,061,292,313,091.00","145,179,102,049,220.00","116,078,602,526,532.00","67,896,584,234,435.00"


In [29]:

Exchange_BCTC = BCTC.groupby(['Exchange','Year_Q'])[["TOTAL ASSETS (280=100+200)","B. OWNER'S EQUITY (400=410+430)","1. Owner's capital ",'15. Profit before tax (50=30+40+41)','18. Net profit after tax  (60 = 50 - 51 - 52)','Market_cap']].sum()
Exchange_BCTC['EPS'] = Exchange_BCTC['18. Net profit after tax  (60 = 50 - 51 - 52)']/(Exchange_BCTC["1. Owner's capital "]/10000)
Exchange_BCTC['PE_quarter'] = Exchange_BCTC['Market_cap']/(Exchange_BCTC['18. Net profit after tax  (60 = 50 - 51 - 52)'])
Exchange_BCTC['Rolling_Profit_4Q'] = Exchange_BCTC.groupby('Exchange')['18. Net profit after tax  (60 = 50 - 51 - 52)'].rolling(4).sum().reset_index(level=0, drop=True)
Exchange_BCTC['Last_Year_Rolling_Profit_4Q'] = Exchange_BCTC.groupby(['Exchange'])['Rolling_Profit_4Q'].shift(4)  #lay loi nhuan rolling  cung ky nam truoc
Exchange_BCTC['YoY_Growth_Rolling_Profit_4Q'] = (Exchange_BCTC['Rolling_Profit_4Q']/Exchange_BCTC['Last_Year_Rolling_Profit_4Q']-1)*100
Exchange_BCTC['ROE(%)'] = Exchange_BCTC['Rolling_Profit_4Q']*100/Exchange_BCTC["1. Owner's capital "]
Exchange_BCTC['ROA(%)'] = Exchange_BCTC['Rolling_Profit_4Q']*100/Exchange_BCTC["TOTAL ASSETS (280=100+200)"]
Exchange_BCTC["Leverage"] = Exchange_BCTC["TOTAL ASSETS (280=100+200)"]/Exchange_BCTC["B. OWNER'S EQUITY (400=410+430)"]
Exchange_BCTC['Rolling_EPS_4Q'] = Exchange_BCTC.groupby('Exchange')['EPS'].rolling(4).sum().reset_index(level=0, drop=True)
Exchange_BCTC['Rolling_PE_4Q'] = Exchange_BCTC['Market_cap']/Exchange_BCTC['Rolling_Profit_4Q']
Exchange_BCTC

TOTAL ASSETS (280=100+200)  B. OWNER'S EQUITY (400=410+430)  \
Exchange Year_Q                                                                
HNX      2022Q1      663,873,396,686,762.00           213,560,249,460,540.00   
         2022Q2      673,732,816,880,137.00           215,997,026,588,739.00   
         2022Q3      679,192,290,945,222.00           220,427,913,266,414.00   
         2022Q4      701,310,886,809,281.00           225,971,574,483,550.00   
         2023Q1      701,313,648,175,627.00           228,300,014,208,221.00   
         2023Q2      704,917,168,995,262.00           227,907,497,989,355.00   
         2023Q3      745,819,821,049,330.00           242,078,494,289,033.00   
         2023Q4      768,885,982,070,875.00           246,399,072,376,082.00   
         2024Q1      770,586,723,025,712.00           251,386,959,411,251.00   
         2024Q2      357,308,768,942,065.00           120,341,096,025,548.00   
HOSE     2022Q1   14,651,826,458,259,566.00         2,508,463,162,960,834.00   
         2022Q2   15,301,400,237,432,612.00         2,583,556,665,374,741.00   
         2022Q3   15,685,771,912,209,656.00         2,667,793,512,143,241.00   
         2022Q4   16,307,603,148,228,428.00         2,718,715,710,984,809.00   
         2023Q1   16,663,463,786,584,980.00         2,797,510,024,373,828.00   
         2023Q2   16,928,654,410,260,620.00         2,853,837,437,247,078.00   
         2023Q3   17,253,804,929,636,742.00         2,915,987,481,316,900.00   
         2023Q4   18,392,993,003,093,656.00         3,011,169,390,540,611.00   
         2024Q1   18,561,584,916,060,008.00         3,111,192,062,182,978.00   
         2024Q2    9,607,698,120,439,536.00         1,725,362,805,757,396.00   
UPCOM    2022Q1    1,715,778,008,250,279.00           569,848,401,143,625.00   
         2022Q2    1,756,336,112,485,612.00           584,118,696,240,530.00   
         2022Q3    1,748,567,501,974,682.00           591,252,480,750,402.00   
         2022Q4    1,729,650,713,758,623.00           591,465,032,311,226.00   
         2023Q1    1,688,944,212,250,660.00           596,618,676,715,006.00   
         2023Q2    1,751,070,003,749,481.00           595,361,893,975,017.00   
         2023Q3    1,791,364,772,894,199.00           614,058,661,405,029.00   
         2023Q4    1,868,658,541,835,595.00           623,171,971,135,913.00   
         2024Q1    1,842,814,372,190,083.00           632,109,063,348,757.00   
         2024Q2    1,016,841,357,712,189.00           306,937,624,151,540.00   

                     1. Owner's capital   15. Profit before tax (50=30+40+41)  \
Exchange Year_Q                                                                 
HNX      2022Q1   128,555,038,621,729.00                 7,216,462,299,893.00   
         2022Q2   133,421,873,420,000.00                 6,455,456,774,879.00   
         2022Q3   140,022,487,073,000.00                 4,613,336,530,290.00   
         2022Q4   143,267,213,799,157.00                 5,828,427,622,700.00   
         2023Q1   143,217,787,910,000.00                 4,874,058,245,684.00   
         2023Q2   143,820,079,380,000.00                 6,218,255,708,156.00   
         2023Q3   156,048,368,050,800.00                 4,626,003,397,309.00   
         2023Q4   157,762,416,520,800.00                 6,641,833,958,970.00   
         2024Q1   158,308,247,459,502.00                 7,008,886,959,760.00   
         2024Q2    75,665,387,490,000.00                 3,951,248,128,830.00   
HOSE     2022Q1 1,267,898,611,797,831.00               138,514,978,954,291.00   
         2022Q2 1,316,166,288,767,831.00               120,945,237,565,462.00   
         2022Q3 1,355,044,061,087,831.00               125,237,179,448,302.00   
         2022Q4 1,415,118,821,024,242.00                93,184,148,115,606.00   
         2023Q1 1,419,779,487,537,832.00               118,101,679,784,116.00   
         2023Q2 1,439,327,682,987,831.00               121,143,043,023,949.00  

In [30]:
BCTC_none_financials_realestate_airline.groupby(['Year_Q'])[['3. Net revenue  (10 = 01 - 03)','5. Gross profit (20 = 10 - 11)','6. Financial income','7. Financial expenses','11. Operating profit {30 = 20 + (21 - 22) - (24 + 25)}','15. Profit before tax (50=30+40+41)','18. Net profit after tax  (60 = 50 - 51 - 52)']].sum()

,3. Net revenue (10 = 01 - 03),5. Gross profit (20 = 10 - 11),6. Financial income,7. Financial expenses,11. Operating profit {30 = 20 + (21 - 22) - (24 + 25)},15. Profit before tax (50=30+40+41),18. Net profit after tax (60 = 50 - 51 - 52)
Year_Q,,,,,,,
2022Q1,"804,672,209,489,280.00","132,254,052,932,955.00","16,644,990,323,055.00","16,945,961,167,358.00","76,452,015,554,746.00","78,225,659,969,670.00","67,299,283,025,897.00"
2022Q2,"895,667,064,067,646.00","146,248,965,372,068.00","23,062,440,786,741.00","25,266,091,546,327.00","81,141,348,752,219.00","82,988,909,216,484.00","70,819,746,739,935.00"
2022Q3,"829,356,007,319,131.00","119,361,773,667,651.00","16,893,127,214,820.00","22,817,791,455,356.00","51,459,139,940,427.00","51,860,794,635,903.00","42,073,154,359,474.00"
2022Q4,"862,112,340,524,896.00","119,671,021,851,910.00","20,633,626,329,099.00","25,956,486,320,637.00","39,600,110,942,522.00","40,588,616,799,144.00","31,674,838,016,723.00"
2023Q1,"722,621,371,547,772.00","104,834,130,340,201.00","18,289,262,420,935.00","22,831,519,723,429.00","43,089,393,425,383.00","44,731,858,516,378.00","36,181,917,897,482.00"
2023Q2,"782,143,667,416,672.00","113,386,193,391,855.00","23,005,565,287,139.00","23,809,469,719,570.00","50,167,390,581,582.00","53,089,020,038,368.00","43,173,518,668,818.00"
2023Q3,"784,547,837,184,593.00","118,593,230,807,515.00","20,536,615,330,304.00","25,284,353,586,198.00","48,669,010,358,089.00","49,784,703,796,504.00","40,560,807,318,462.00"
2023Q4,"855,982,411,210,212.00","124,608,459,291,088.00","21,042,111,773,057.00","21,299,550,395,322.00","50,013,614,488,287.00","54,509,425,385,004.00","44,697,436,176,411.00"
2024Q1,"770,360,515,912,058.00","115,064,863,119,820.00","16,689,869,370,447.00","19,915,259,680,173.00","50,098,606,156,881.00","51,544,614,239,541.00","42,033,988,254,882.00"


TRẢ LỜI CHO CÂU HỎI THỊ TRƯỜNG NONE BANK, REAL ESTATE, AIRLINE

In [31]:
BCTC_none_financials_realestate_airline['Market_cap'] = BCTC_none_financials_realestate_airline["1. Owner's capital "]*BCTC_none_financials_realestate_airline['Close Price End of period']/10000
Exchange_BCTC_None = BCTC_none_financials_realestate_airline.groupby(['Exchange','Year_Q'])[["TOTAL ASSETS (280=100+200)","B. OWNER'S EQUITY (400=410+430)","1. Owner's capital ","III. Short-term receivables","IV. Inventories",'15. Profit before tax (50=30+40+41)','18. Net profit after tax  (60 = 50 - 51 - 52)','EBITDA','Market_cap',"Net cash flows from operating activities","Net cash flows from investing activities","Net cash flows from financing activities"]].sum()
Exchange_BCTC_None['EPS'] = Exchange_BCTC_None['18. Net profit after tax  (60 = 50 - 51 - 52)']/(Exchange_BCTC_None["1. Owner's capital "]/10000)
Exchange_BCTC_None['PE_quarter'] = Exchange_BCTC_None['Market_cap']/(Exchange_BCTC_None['18. Net profit after tax  (60 = 50 - 51 - 52)'])
Exchange_BCTC_None['Rolling_Profit_4Q'] = Exchange_BCTC_None.groupby('Exchange')['18. Net profit after tax  (60 = 50 - 51 - 52)'].rolling(4).sum().reset_index(level=0, drop=True)
Exchange_BCTC_None['Rolling_PE_4Q'] = Exchange_BCTC_None['Market_cap']/Exchange_BCTC_None['Rolling_Profit_4Q']
Exchange_BCTC_None['Last_Year_Rolling_Profit_4Q'] = Exchange_BCTC_None.groupby(['Exchange'])['Rolling_Profit_4Q'].shift(4)  #lay loi nhuan rolling  cung ky nam truoc
Exchange_BCTC_None['YoY_Growth_Rolling_Profit_4Q'] = (Exchange_BCTC_None['Rolling_Profit_4Q']/Exchange_BCTC_None['Last_Year_Rolling_Profit_4Q']-1)*100
Exchange_BCTC_None['ROE(%)'] = Exchange_BCTC_None['Rolling_Profit_4Q']*100/Exchange_BCTC_None["1. Owner's capital "]
Exchange_BCTC_None['ROA(%)'] = Exchange_BCTC_None['Rolling_Profit_4Q']*100/Exchange_BCTC_None["TOTAL ASSETS (280=100+200)"]
Exchange_BCTC_None["Leverage"] = Exchange_BCTC_None["TOTAL ASSETS (280=100+200)"]/Exchange_BCTC_None["B. OWNER'S EQUITY (400=410+430)"]

Exchange_BCTC_None

C:\Users\khoi.ln\AppData\Local\Temp\ipykernel_10136\1738155496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BCTC_none_financials_realestate_airline['Market_cap'] = BCTC_none_financials_realestate_airline["1. Owner's capital "]*BCTC_none_financials_realestate_airline['Close Price End of period']/10000


TOTAL ASSETS (280=100+200)  B. OWNER'S EQUITY (400=410+430)  \
Exchange Year_Q                                                                
HNX      2022Q1      305,004,353,047,220.00           130,274,688,979,608.00   
         2022Q2      313,392,344,997,258.00           129,494,514,240,989.00   
         2022Q3      316,572,856,838,791.00           132,610,540,679,967.00   
         2022Q4      319,924,501,308,795.00           135,052,378,477,062.00   
         2023Q1      320,564,831,265,583.00           136,259,275,564,649.00   
         2023Q2      321,901,210,514,897.00           135,883,924,668,020.00   
         2023Q3      333,627,443,611,873.00           145,438,874,822,794.00   
         2023Q4      339,159,057,230,070.00           147,597,029,871,731.00   
         2024Q1      337,150,778,816,835.00           149,864,838,545,205.00   
         2024Q2      148,975,114,122,815.00            68,928,805,482,226.00   
HOSE     2022Q1    2,055,460,456,047,898.00           955,861,357,656,356.00   
         2022Q2    2,110,688,300,029,010.00           981,545,502,149,470.00   
         2022Q3    2,091,744,449,801,499.00           987,108,223,131,394.00   
         2022Q4    2,075,846,564,074,925.00           997,472,011,016,235.00   
         2023Q1    2,071,019,692,264,288.00         1,013,999,756,483,527.00   
         2023Q2    2,117,258,172,028,785.00         1,024,953,392,709,564.00   
         2023Q3    2,103,311,357,991,108.00         1,014,422,835,452,167.00   
         2023Q4    2,123,641,684,968,913.00         1,013,110,504,562,205.00   
         2024Q1    2,158,451,644,507,790.00         1,041,764,746,891,985.00   
         2024Q2    1,046,520,820,662,214.00           494,377,266,762,434.00   
UPCOM    2022Q1    1,027,276,699,660,831.00           477,086,857,697,256.00   
         2022Q2    1,049,894,573,861,219.00           491,592,038,831,554.00   
         2022Q3    1,034,065,078,704,828.00           497,329,843,469,858.00   
         2022Q4    1,024,966,431,695,495.00           503,360,813,092,064.00   
         2023Q1    1,005,627,454,020,026.00           506,519,557,158,069.00   
         2023Q2    1,020,731,550,285,038.00           504,156,277,145,538.00   
         2023Q3    1,047,404,484,900,467.00           514,420,894,033,812.00   
         2023Q4    1,054,146,033,372,837.00           517,414,322,630,871.00   
         2024Q1    1,051,501,055,260,029.00           524,699,590,129,326.00   
         2024Q2      458,881,360,499,550.00           249,471,407,132,892.00   

                   1. Owner's capital   III. Short-term receivables  \
Exchange Year_Q                                                       
HNX      2022Q1  78,857,745,651,729.00        82,149,940,453,807.00   
         2022Q2  80,188,895,700,000.00        87,028,498,105,553.00   
         2022Q3  82,246,514,733,000.00        89,913,308,246,762.00   
         2022Q4  82,912,646,969,157.00        92,544,095,917,804.00   
         2023Q1  82,814,195,320,000.00        90,847,953,300,445.00   
         2023Q2  83,837,420,300,000.00        92,203,237,624,054.00   
         2023Q3  90,746,804,190,800.00        95,369,067,441,693.00   
         2023Q4  91,580,229,560,800.00        97,164,193,159,058.00   
         2024Q1  91,500,990,499,502.00        95,246,857,613,253.00   
         2024Q2  41,510,177,150,000.00        43,179,214,240,721.00   
HOSE     2022Q1 475,146,343,787,831.00       340,576,981,216,917.00   
         2022Q2 499,870,113,107,831.00       356,665,404,832,635.00   
         2022Q3 518,739,824,147,831.00       351,646,529,107,853.00   
         2022Q4 524,787,579,384,242.00       342,963,341,206,023.00   
         2023Q1 526,344,760,887,831.00       368,381,820,330,056.00   
         2023Q2 528,395,593,477,831.00       376,141,948,379,914.00   
         2023Q3 533,217,053,357,831.00       355,401,825,590,118.00   
         2023Q4 537,647,715,907,810.00       326,498,404,224,107.00   
         2024Q1 538,217,669,317,831.

ĐÁNH GIÁ PE THEO NGÀNH

In [32]:
Sector_BCTC = BCTC.groupby(['VS-Sector L4','Year_Q'])[['Stock code',"TOTAL ASSETS (280=100+200)","B. OWNER'S EQUITY (400=410+430)","1. Owner's capital ","III. Short-term receivables","IV. Inventories","3. Net revenue  (10 = 01 - 03)","5. Gross profit (20 = 10 - 11)",'EBITDA',"7. Financial expenses","9. Selling expenses","10. General and administrative expenses",'11. Operating profit {30 = 20 + (21 - 22) - (24 + 25)}','18. Net profit after tax  (60 = 50 - 51 - 52)','Market_cap',"Net cash flows from operating activities","Net cash flows from investing activities","Net cash flows from financing activities"]].sum().reset_index()

path_bctc_clean_quarterly = '../../data/clean/vietstock_data/bctc_hsx_hnx_upcom/quarterly/'
Sector_BCTC.to_excel(path_bctc_clean_quarterly+"Sector_BCTC.xlsx", merge_cells=True) # merge_cells=True giúp xuất các cột bị Merge như VS-Sector L4, và Year_Q
Sector_BCTC
# Nhóm dữ liệu theo 'VS-Sector L4' và 'Year_Q'


,VS-Sector L4,Year_Q,Stock code,TOTAL ASSETS (280=100+200),B. OWNER'S EQUITY (400=410+430),1. Owner's capital,III. Short-term receivables,IV. Inventories,3. Net revenue (10 = 01 - 03),5. Gross profit (20 = 10 - 11),EBITDA,7. Financial expenses,9. Selling expenses,10. General and administrative expenses,11. Operating profit {30 = 20 + (21 - 22) - (24 + 25)},18. Net profit after tax (60 = 50 - 51 - 52),Market_cap,Net cash flows from operating activities,Net cash flows from investing activities,Net cash flows from financing activities
0,Advertising,2022Q1,ADG,"457,091,771,205.00","273,773,948,171.00","198,890,980,000.00","97,954,113,334.00","16,052,188,574.00","117,827,295,665.00","11,256,448,978.00","3,316,799,791.00","1,152,531,860.00","6,347,534,837.00","5,041,410,061.00","4,474,050,184.00","2,222,548,433.00","1,012,355,088,200.00","-33,455,127,514.00","10,372,679,205.00","31,577,742,319.00"
1,Advertising,2022Q2,ADG,"484,922,824,670.00","304,358,343,125.00","198,890,980,000.00","136,821,715,385.00","19,650,041,179.00","128,813,005,060.00","28,609,176,860.00","23,748,084,392.00","2,138,620,791.00","6,307,466,547.00","6,119,478,505.00","19,529,385,614.00","22,955,915,437.00","749,818,994,600.00","5,577,620,649.00","-12,301,833,245.00","7,458,318,504.00"
2,Advertising,2022Q3,ADG,"471,246,245,883.00","299,264,031,643.00","198,890,980,000.00","147,989,031,287.00","13,020,703,554.00","126,423,874,065.00","13,864,834,938.00","3,268,979,772.00","2,174,208,679.00","5,705,210,386.00","6,707,525,388.00","5,593,368,190.00","3,826,429,647.00","837,331,025,800.00","17,913,353,542.00","-11,425,088,869.00","-9,299,873,479.00"
3,Advertising,2022Q4,ADG,"412,298,810,824.00","327,371,086,584.00","213,805,210,000.00","118,255,020,718.00","1,401,933,598.00","164,128,089,781.00","41,979,085,945.00","29,894,952,743.00","4,099,185,191.00","4,141,877,088.00","6,629,728,920.00","33,882,107,553.00","26,103,098,052.00","502,442,243,500.00","25,118,703,678.00","79,675,031,745.00","-86,996,975,106.00"
4,Advertising,2023Q1,ADG,"411,516,833,190.00","324,144,982,707.00","213,805,210,000.00","88,056,788,610.00","9,981,035,242.00","82,229,579,820.00","11,782,581,897.00","-1,773,894,942.00","774,112,408.00","7,044,216,414.00","8,928,002,703.00","-1,122,323,515.00","-1,615,948,865.00","812,459,798,000.00","17,010,767,976.00","-43,459,209,873.00","15,182,651,980.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,Wireless Telecommunication Services,2023Q1,VGI,"50,502,804,475,633.00","29,690,953,206,043.00","30,438,112,000,000.00","14,981,026,894,760.00","1,951,143,989,365.00","6,481,102,464,663.00","3,247,601,469,143.00","858,016,509,361.00","906,187,839,061.00","761,262,334,570.00","1,335,606,090,668.00","959,198,943,080.00","594,542,412,778.00","64,833,178,560,000.00","2,864,186,726,881.00","-1,154,996,431,978.00","-921,206,772,603.00"
858,Wireless Telecommunication Services,2023Q2,VGI,"48,424,867,370,960.00","28,750,124,791,181.00","30,438,112,000,000.00","12,511,301,384,221.00","2,453,567,719,032.00","6,861,419,133,040.00","3,138,944,970,744.00","-1,262,333,589,000.00","1,257,479,389,190.00","783,410,690,179.00","2,701,183,624,335.00","-811,793,476,997.00","-1,219,197,126,097.00","71,225,182,080,000.00","2,329,928,875,526.00","-940,627,565,132.00","-1,141,710,897,621.00"
859,Wireless Telecommunication Services,2023Q3,VGI,"50,803,353,975,670.00","29,914,708,205,697.00","30,438,112,000,000.00","11,943,237,946,185.00","2,827,815,035,891.00","7,325,551,939,115.00","3,749,958,955,757.00","1,983,950,432,695.00","859,055,376,619.00","1,004,279,680,321.00","1,518,104,994,957.00","2,098,074,484,066.00","1,409,588,119,101.00","70,312,038,720,000.00","5,230,127,845,525.00","-3,292,088,315,583.00","-1,115,463,343,252.00"
860,Wireless Telecommunication Services,2023Q4,VGI,"52,371,351,113,757.00","30,362,756,269,123.00","30,438,112,000,000.00","10,944,320,309,111.00","2,514,299,769,920.00","7,563,571,969,411.00","4,130,892,813,867.00",

In [33]:
# Sắp xếp DataFrame theo 'VS-Sector L4' và 'Year_Q'
BCTC_sorted = BCTC.sort_values(['VS-Sector L4', 'Year_Q'])

# Nhóm dữ liệu theo 'VS-Sector L4' và 'Year_Q'
grouped = BCTC_sorted.groupby(['VS-Sector L4', 'Year_Q'])['Stock code'].apply(list).reset_index()


# Tạo một hàm để tìm các 'stock code' bị thiếu so với quý trước
def find_missing_codes(group):
    prev_stock_codes = None
    missing_codes_list = []
    for index, row in group.iterrows():
        current_stock_codes = set(row['Stock code'])
        if prev_stock_codes is None:
            missing_codes_list.append('')
        else:
            missing_codes = prev_stock_codes - current_stock_codes
            missing_codes_list.append(','.join(missing_codes))
        prev_stock_codes = current_stock_codes
    group['Missing Stock Codes'] = missing_codes_list
    return group

# Áp dụng hàm cho mỗi nhóm 'VS-Sector L4'
missing_codes = grouped.groupby('VS-Sector L4').apply(find_missing_codes).reset_index(drop=True)
missing_codes
Sector_BCTC = Sector_BCTC.merge(
    missing_codes[['VS-Sector L4', 'Year_Q', 'Missing Stock Codes']],
    on=['VS-Sector L4', 'Year_Q'],
    how='left'
)
Sector_BCTC

,VS-Sector L4,Year_Q,Stock code,TOTAL ASSETS (280=100+200),B. OWNER'S EQUITY (400=410+430),1. Owner's capital,III. Short-term receivables,IV. Inventories,3. Net revenue (10 = 01 - 03),5. Gross profit (20 = 10 - 11),EBITDA,7. Financial expenses,9. Selling expenses,10. General and administrative expenses,11. Operating profit {30 = 20 + (21 - 22) - (24 + 25)},18. Net profit after tax (60 = 50 - 51 - 52),Market_cap,Net cash flows from operating activities,Net cash flows from investing activities,Net cash flows from financing activities,Missing Stock Codes
0,Advertising,2022Q1,ADG,"457,091,771,205.00","273,773,948,171.00","198,890,980,000.00","97,954,113,334.00","16,052,188,574.00","117,827,295,665.00","11,256,448,978.00","3,316,799,791.00","1,152,531,860.00","6,347,534,837.00","5,041,410,061.00","4,474,050,184.00","2,222,548,433.00","1,012,355,088,200.00","-33,455,127,514.00","10,372,679,205.00","31,577,742,319.00",
1,Advertising,2022Q2,ADG,"484,922,824,670.00","304,358,343,125.00","198,890,980,000.00","136,821,715,385.00","19,650,041,179.00","128,813,005,060.00","28,609,176,860.00","23,748,084,392.00","2,138,620,791.00","6,307,466,547.00","6,119,478,505.00","19,529,385,614.00","22,955,915,437.00","749,818,994,600.00","5,577,620,649.00","-12,301,833,245.00","7,458,318,504.00",
2,Advertising,2022Q3,ADG,"471,246,245,883.00","299,264,031,643.00","198,890,980,000.00","147,989,031,287.00","13,020,703,554.00","126,423,874,065.00","13,864,834,938.00","3,268,979,772.00","2,174,208,679.00","5,705,210,386.00","6,707,525,388.00","5,593,368,190.00","3,826,429,647.00","837,331,025,800.00","17,913,353,542.00","-11,425,088,869.00","-9,299,873,479.00",
3,Advertising,2022Q4,ADG,"412,298,810,824.00","327,371,086,584.00","213,805,210,000.00","118,255,020,718.00","1,401,933,598.00","164,128,089,781.00","41,979,085,945.00","29,894,952,743.00","4,099,185,191.00","4,141,877,088.00","6,629,728,920.00","33,882,107,553.00","26,103,098,052.00","502,442,243,500.00","25,118,703,678.00","79,675,031,745.00","-86,996,975,106.00",
4,Advertising,2023Q1,ADG,"411,516,833,190.00","324,144,982,707.00","213,805,210,000.00","88,056,788,610.00","9,981,035,242.00","82,229,579,820.00","11,782,581,897.00","-1,773,894,942.00","774,112,408.00","7,044,216,414.00","8,928,002,703.00","-1,122,323,515.00","-1,615,948,865.00","812,459,798,000.00","17,010,767,976.00","-43,459,209,873.00","15,182,651,980.00",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,Wireless Telecommunication Services,2023Q1,VGI,"50,502,804,475,633.00","29,690,953,206,043.00","30,438,112,000,000.00","14,981,026,894,760.00","1,951,143,989,365.00","6,481,102,464,663.00","3,247,601,469,143.00","858,016,509,361.00","906,187,839,061.00","761,262,334,570.00","1,335,606,090,668.00","959,198,943,080.00","594,542,412,778.00","64,833,178,560,000.00","2,864,186,726,881.00","-1,154,996,431,978.00","-921,206,772,603.00",
858,Wireless Telecommunication Services,2023Q2,VGI,"48,424,867,370,960.00","28,750,124,791,181.00","30,438,112,000,000.00","12,511,301,384,221.00","2,453,567,719,032.00","6,861,419,133,040.00","3,138,944,970,744.00","-1,262,333,589,000.00","1,257,479,389,190.00","783,410,690,179.00","2,701,183,624,335.00","-811,793,476,997.00","-1,219,197,126,097.00","71,225,182,080,000.00","2,329,928,875,526.00","-940,627,565,132.00","-1,141,710,897,621.00",
859,Wireless Telecommunication Services,2023Q3,VGI,"50,803,353,975,670.00","29,914,708,205,697.00","30,438,112,000,000.00","11,943,237,946,185.00","2,827,815,035,891.00","7,325,551,939,115.00","3,749,958,955,757.00","1,983,950,432,695.00","859,055,376,619.00","1,004,279,680,321.00","1,518,104,994,957.00","2,098,074,484,066.00","1,409,588,119,101.00","70,312,038,720,000.00","5,230,127,845,525.00","-3,292,088,315,583.00","-1,115,463,343,252.00",
860,Wireless Telecommunication Services,2023Q4,VGI,"52,371,351,113,757.00","30,362,756,269,123.00","30,438,112,000,000.00","10,944,320,309,111.00","2,514,299,769,920.00","7,563,571,969

Chọn Sector

In [34]:
Choose_sector = "Textiles"
Choose_sector_overview = Sector_BCTC[Sector_BCTC['VS-Sector L4'] == Choose_sector]
Choose_sector_overview

,VS-Sector L4,Year_Q,Stock code,TOTAL ASSETS (280=100+200),B. OWNER'S EQUITY (400=410+430),1. Owner's capital,III. Short-term receivables,IV. Inventories,3. Net revenue (10 = 01 - 03),5. Gross profit (20 = 10 - 11),EBITDA,7. Financial expenses,9. Selling expenses,10. General and administrative expenses,11. Operating profit {30 = 20 + (21 - 22) - (24 + 25)},18. Net profit after tax (60 = 50 - 51 - 52),Market_cap,Net cash flows from operating activities,Net cash flows from investing activities,Net cash flows from financing activities,Missing Stock Codes
803,Textiles,2022Q1,ADSFTMHTGKMRNDTNTTPPHSTKSVDTVT,"16,702,680,151,866.00","6,641,255,157,098.00","3,849,737,280,000.00","3,920,487,306,510.00","3,602,306,704,466.00","4,155,098,551,474.00","565,299,695,419.00","330,743,191,960.00","88,552,865,899.00","84,480,857,352.00","139,240,761,602.00","436,011,152,573.00","371,950,281,558.00","12,136,470,711,130.00","691,534,954,211.00","-478,280,941,708.00","16,524,302,850.00",
804,Textiles,2022Q2,ADSFTMHTGKMRNDTNTTPPHSTKSVDTVT,"17,226,778,397,989.00","6,651,950,775,771.00","4,042,518,030,000.00","3,580,687,313,970.00","4,051,104,692,311.00","3,960,953,730,486.00","548,252,139,074.00","491,805,758,377.00","139,617,160,542.00","80,598,136,649.00","139,502,167,123.00","381,170,128,225.00","315,330,847,038.00","9,354,139,501,390.00","50,377,758,590.00","14,591,746,330.00","128,498,877,304.00",
805,Textiles,2022Q3,ADSFTMHTGKMRNDTNTTPPHSTKSVDTVT,"17,196,707,301,687.00","6,950,481,199,596.00","4,256,382,440,000.00","3,518,868,740,744.00","3,881,026,869,486.00","4,083,968,618,239.00","449,836,148,733.00","116,822,257,141.00","167,719,350,756.00","78,127,122,216.00","166,596,984,775.00","189,683,627,271.00","139,841,322,109.00","8,473,909,999,700.00","302,238,174,482.00","-39,894,020,925.00","-159,028,246,372.00",
806,Textiles,2022Q4,ADSFTMHTGKMRNDTNTTPPHSTKSVDTVT,"16,885,741,373,176.00","6,910,817,169,287.00","4,274,441,520,000.00","3,470,683,138,236.00","4,240,189,913,279.00","3,618,314,619,597.00","360,991,136,074.00","275,726,745,982.00","214,124,216,303.00","64,739,027,112.00","187,988,316,343.00","129,155,447,795.00","115,426,816,260.00","6,166,175,816,880.00","-102,166,387,664.00","-276,055,832,589.00","-181,718,522,175.00",
807,Textiles,2023Q1,ADSFTMHTGKMRNDTNTTPPHSTKSVDTVT,"16,267,607,133,359.00","6,664,649,611,748.00","4,344,441,520,000.00","3,491,925,177,130.00","3,822,469,102,974.00","3,383,336,238,338.00","335,390,292,270.00","59,844,225,019.00","130,537,239,290.00","53,824,970,017.00","151,011,356,296.00","174,071,809,456.00","129,383,606,809.00","6,782,690,086,460.00","-67,028,462,024.00","129,534,726,583.00","-41,590,931,169.00",
808,Textiles,2023Q2,ADSFTMHTGKMRNDTNTTPPHSTKSVDTVT,"16,236,071,482,977.00","6,381,726,203,952.00","4,404,437,850,000.00","3,454,463,394,781.00","4,028,428,825,928.00","3,823,980,642,604.00","293,803,579,658.00","170,835,355,716.00","131,032,071,606.00","68,156,415,781.00","160,198,091,397.00","80,447,717,841.00","47,967,407,055.00","7,247,730,713,000.00","508,632,805,697.00","-244,237,473,020.00","-501,835,234,276.00",
809,Textiles,2023Q3,ADSFTMHTGKMRNDTNTTPPHSTKSVDTVT,"16,878,635,913,337.00","6,567,517,355,578.00","4,603,325,770,000.00","4,161,045,402,828.00","3,713,712,390,933.00","3,809,229,426,266.00","413,706,180,489.00","29,492,913,483.00","192,571,844,873.00","72,548,953,056.00","187,763,653,389.00","105,481,770,361.00","71,307,176,440.00","8,088,485,794,760.00","234,387,158,146.00","-357,879,094,093.00","171,213,313,297.00",
810,Textiles,2023Q4,ADSFTMHTGKMRNDTNTTPPHSTKSVDTVT,"17,337,725,657,812.00","6,578,836,425,643.00","4,603,325,770,000.00","3,473,293,569,779.00","4,014,440,695,850.00","3,556,536,768,222.00","327,289,893,073.00","221,517,184,892.00","147,416,756,994.00","69,012,102,878.00","181,653,110,400.00","72,787,952,083.00","42,753,769,577.00","7,141,577,177,490.00","-12,114,785,445.00","-379,766,050,707.00","641,638,789,406.00",
811,Textiles,2024Q1,ADSFTMHTGKMRNDTNTTPPHSTKSVDTVTSPB,"18,5

In [35]:
Choose_sector_detailview = BCTC[BCTC['VS-Sector L4']==Choose_sector][["Stock code",'Exchange','Company name',"Year_Q","TOTAL ASSETS (280=100+200)","1. Owner's capital ",'18. Net profit after tax  (60 = 50 - 51 - 52)','Market_cap']]

Choose_sector_detailview

,Stock code,Exchange,Company name,Year_Q,TOTAL ASSETS (280=100+200),1. Owner's capital,18. Net profit after tax (60 = 50 - 51 - 52),Market_cap
204,ADS,HOSE,CTCP Damsan,2022Q1,"2,289,053,692,567.00","380,694,500,000.00","28,528,753,613.00","1,406,666,177,500.00"
205,ADS,HOSE,CTCP Damsan,2022Q2,"2,047,686,182,980.00","380,694,500,000.00","17,868,090,092.00","742,354,275,000.00"
206,ADS,HOSE,CTCP Damsan,2022Q3,"2,063,676,912,352.00","437,790,340,000.00","20,520,849,031.00","599,772,765,800.00"
207,ADS,HOSE,CTCP Damsan,2022Q4,"2,209,819,125,112.00","437,790,340,000.00","4,306,290,420.00","411,960,709,940.00"
208,ADS,HOSE,CTCP Damsan,2023Q1,"2,271,739,248,958.00","507,790,340,000.00","13,148,690,425.00","629,660,021,600.00"
209,ADS,HOSE,CTCP Damsan,2023Q2,"2,328,844,466,071.00","507,790,340,000.00","24,810,633,830.00","870,860,433,100.00"
210,ADS,HOSE,CTCP Damsan,2023Q3,"2,640,505,061,312.00","583,947,270,000.00","30,592,809,910.00","835,044,596,100.00"
211,ADS,HOSE,CTCP Damsan,2023Q4,"2,535,817,472,504.00","583,947,270,000.00","8,390,053,849.00","770,810,396,400.00"
212,ADS,HOSE,CTCP Damsan,2024Q1,"2,534,075,432,615.00","733,947,270,000.00","8,318,605,312.00","1,097,251,168,650.00"
213,ADS,HOSE,CTCP Damsan,2024Q2,"2,496,648,484,855.00","763,947,270,000.00","6,588,699,823.00","859,440,678,750.00"


Tìm các cp đã có bctc 2024Q2, sau đó lấy bctc Q1,Q2 của 2024 và tính EBITDA growth

In [36]:
# Tìm các 'stock code' có 'Year_Q' là '2024Q2' và 
valid_stock_codes = BCTC[BCTC['Year_Q'] == '2024Q2']['Stock code'].unique()

# Lọc các hàng dựa trên các 'stock code' đã tìm thấy
BCTC_2024Q2_filtered = BCTC[BCTC['Stock code'].isin(valid_stock_codes)]
BCTC_2024Q2_filtered = BCTC_2024Q2_filtered[(BCTC_2024Q2_filtered['Year'] >= 2023) & (BCTC_2024Q2_filtered['Quarter'] <= 2) ]
BCTC_2024Q2_filtered
# Đảm bảo DataFrame được sắp xếp theo 'Stock code', 'Year' và 'Quarter'
BCTC_2024Q2_filtered = BCTC_2024Q2_filtered.sort_values(by=['Stock code', 'Year', 'Quarter'])
# Tính toán sự tăng trưởng revenue so với cùng kỳ
BCTC_2024Q2_filtered['EBITDA_growth'] = BCTC_2024Q2_filtered.groupby(['Stock code', 'Quarter'])['EBITDA'].pct_change() * 100
BCTC_2024Q2_filtered

C:\Users\khoi.ln\AppData\Local\Temp\ipykernel_10136\3750493269.py:11: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  BCTC_2024Q2_filtered['EBITDA_growth'] = BCTC_2024Q2_filtered.groupby(['Stock code', 'Quarter'])['EBITDA'].pct_change() * 100


,No.,Stock code,Company name,Exchange,Full Name Vietnamese,Sector (NAICS) Level 1,Sector (NAICS) Level 2,Sector (NAICS) Level 3,Year,Quarter,Year_Q,A. SHORT-TERM ASSETS (100=110+120+130+140+150),I. Cash and cash equivalents,1. Cash,2. Cash equivalents,II. Short-term financial investments,1. Available for sale securities,2. Provision for diminution in value of available for sale securities (*),3. Held to maturity investments,III. Short-term receivables,1. Short-term trade accounts receivable,2. Short-term prepayments to suppliers,3. Short-term inter-company receivables,4. Construction contract progress receipts due from customers,5. Short-term loan receivables,6. Other short-term receivables,7. Provision for short-term doubtful debts (*),8. Assets awaiting resolution,IV. Inventories,1. Inventories,2. Provision for decline in value of inventories,V. Other short-term assets,1. Short-term prepayments,2. Value added tax to be reclaimed,3. Taxes and other receivables from state authorities,4. Government bonds,5. Other short-term assets,B. LONG-TERM ASSETS (200=210+220+240+250+260+270),I. Long-term receivables,1. Long-term trade receivables,2. Long-term prepayments to suppliers,3. Capital at inter-company,4. Long-term inter-company receivables,5. Long-term loan receivables,6. Other long-term receivables,7. Provision for long-term doubtful debts,II. Fixed assets,1. Tangible fixed assets,- Cost,- Accumulated depreciation,2. Financial leased fixed assets,- Cost.1,- Accumulated depreciation.1,3. Intangible fixed assets,- Cost.2,- Accumulated depreciation.2,III. Investment properties,- Cost.3,- Accumulated depreciation.3,IV. Long-term assets in progress,1. Long-term production in progress,2. Construction in progress,V. Long-term financial investments,1. Investments in subsidiaries,"2. Investments in associates, joint-ventures",3. Investments in other entities,4. Provision for diminution in value of long-term investments,5. Held to maturity investments,6. Other long-term investments,VI. Other long-term assets,1. Long-term prepayments,2. Deferred income tax assets,"3. Long-term equipment, supplies, spare parts",4. Other long-term assets,VII. Goodwill,TOTAL ASSETS (280=100+200),A. LIABILITIES (300=210+330),I. Short -term liabilities,1. Short-term trade accounts payable,2. Short-term advances from customers,3. Taxes and other payables to state authorities,4. Payable to employees,5. Short-term acrrued expenses,6. Short-term inter-company payables,7. Construction contract progress payments due to suppliers,8. Short-term unearned revenue,9. Other short-term payables,10. Short-term borrowings and financial leases,11. Provision for short-term liabilities,12.. Bonus and welfare fund,13. Price stabilization fund,14. Government bonds,II. Long-term liabilities,1. Long-term trade payables,2. Long-term advances from customers,3. Long-term acrrued expenses,4. Inter-company payables on business capital,5. Long-term inter-company payables,6. Long-term unearned revenue,7. Other long-term liabilities,8. Long-term borrowings and financial leases,9. Convertible bonds,10. Preferred stock (Debts),11. Deferred income tax liabilities,12. Provision for long-term liabilities,13. Fund for technology development,14. Provision for severance allowances,B. OWNER'S EQUITY (400=410+430),I. Owner's equity,1. Owner's capital,- Common stock with voting right,- Preferred stock,2. Share premium,3. Convertible bond option,4. Other capital of owners,5. Treasury shares,6. Assets revaluation differences,7. Foreign exchange differences,8. Investment and development fund,9. Fund to support corporate restructuring,10. Other funds from owner's equity,11. Undistributed earnings after tax,- Accumulated retained earning at the end of the previous period,- Undistributed earnings in this period,12. Reserves for investment in construction,13. Minority's interest,14. Financial reserves,II. Other resources and funds,1. Subsidized not-for-profit funds,2. Funds invested in fixed assets,C. MINORI

Lọc theo EBITDA Growth,và loại các cp có EBITDA 2024Q2 âm

In [41]:
# Lọc các 'Stock code' có revenue âm trong năm 2024
invalid_stock_codes_2024 = BCTC_2024Q2_filtered[(BCTC_2024Q2_filtered['Year_Q'] == '2024Q2') & (BCTC_2024Q2_filtered['EBITDA'] < 0)]['Stock code'].unique()
BCTC_2024Q2_filtered = BCTC_2024Q2_filtered[~BCTC_2024Q2_filtered['Stock code'].isin(invalid_stock_codes_2024)]
Ebitda_G15 =BCTC_2024Q2_filtered[BCTC_2024Q2_filtered['EBITDA_growth'] >10]  # lọc các Stockcode có Ebitda tang hon 20%
# tìm các cp có 2 quý liên tục có EBITDA tang truong 20%
stock_code_ebitda_cond = Ebitda_G15['Stock code'].value_counts()
stock_code_ebitda_cond = stock_code_ebitda_cond[stock_code_ebitda_cond >=2 ].index
Ebitda_G15_2Q = BCTC_2024Q2_filtered[BCTC_2024Q2_filtered['Stock code'].isin(stock_code_ebitda_cond)]
Ebitda_G15_2Q = Ebitda_G15_2Q[['Stock code','Year_Q','3. Net revenue  (10 = 01 - 03)','5. Gross profit (20 = 10 - 11)','Gross_profit_margin','EBITDA','EBITDA_growth','6. Financial income','7. Financial expenses','11. Operating profit {30 = 20 + (21 - 22) - (24 + 25)}','Oprating_profit_margin','15. Profit before tax (50=30+40+41)','18. Net profit after tax  (60 = 50 - 51 - 52)','net_profit_margin','Rolling_Profit_4Q','Rolling_EPS_4Q','Rolling_PE_4Q','ROE(%)','ROA(%)']]
Ebitda_G15_2Q

path_bctc_clean_quarterly = '../../data/clean/vietstock_data/bctc_hsx_hnx_upcom/quarterly/'
Ebitda_G15_2Q.to_excel(path_bctc_clean_quarterly+"Ebitda_G15_2Q.xlsx", merge_cells=True) # merge_cells=True giúp xuất các cột bị Merge như VS-Sector L4, và Year_Q
Sector_BCTC


,VS-Sector L4,Year_Q,Stock code,TOTAL ASSETS (280=100+200),B. OWNER'S EQUITY (400=410+430),1. Owner's capital,III. Short-term receivables,IV. Inventories,3. Net revenue (10 = 01 - 03),5. Gross profit (20 = 10 - 11),EBITDA,7. Financial expenses,9. Selling expenses,10. General and administrative expenses,11. Operating profit {30 = 20 + (21 - 22) - (24 + 25)},18. Net profit after tax (60 = 50 - 51 - 52),Market_cap,Net cash flows from operating activities,Net cash flows from investing activities,Net cash flows from financing activities,Missing Stock Codes
0,Advertising,2022Q1,ADG,"457,091,771,205.00","273,773,948,171.00","198,890,980,000.00","97,954,113,334.00","16,052,188,574.00","117,827,295,665.00","11,256,448,978.00","3,316,799,791.00","1,152,531,860.00","6,347,534,837.00","5,041,410,061.00","4,474,050,184.00","2,222,548,433.00","1,012,355,088,200.00","-33,455,127,514.00","10,372,679,205.00","31,577,742,319.00",
1,Advertising,2022Q2,ADG,"484,922,824,670.00","304,358,343,125.00","198,890,980,000.00","136,821,715,385.00","19,650,041,179.00","128,813,005,060.00","28,609,176,860.00","23,748,084,392.00","2,138,620,791.00","6,307,466,547.00","6,119,478,505.00","19,529,385,614.00","22,955,915,437.00","749,818,994,600.00","5,577,620,649.00","-12,301,833,245.00","7,458,318,504.00",
2,Advertising,2022Q3,ADG,"471,246,245,883.00","299,264,031,643.00","198,890,980,000.00","147,989,031,287.00","13,020,703,554.00","126,423,874,065.00","13,864,834,938.00","3,268,979,772.00","2,174,208,679.00","5,705,210,386.00","6,707,525,388.00","5,593,368,190.00","3,826,429,647.00","837,331,025,800.00","17,913,353,542.00","-11,425,088,869.00","-9,299,873,479.00",
3,Advertising,2022Q4,ADG,"412,298,810,824.00","327,371,086,584.00","213,805,210,000.00","118,255,020,718.00","1,401,933,598.00","164,128,089,781.00","41,979,085,945.00","29,894,952,743.00","4,099,185,191.00","4,141,877,088.00","6,629,728,920.00","33,882,107,553.00","26,103,098,052.00","502,442,243,500.00","25,118,703,678.00","79,675,031,745.00","-86,996,975,106.00",
4,Advertising,2023Q1,ADG,"411,516,833,190.00","324,144,982,707.00","213,805,210,000.00","88,056,788,610.00","9,981,035,242.00","82,229,579,820.00","11,782,581,897.00","-1,773,894,942.00","774,112,408.00","7,044,216,414.00","8,928,002,703.00","-1,122,323,515.00","-1,615,948,865.00","812,459,798,000.00","17,010,767,976.00","-43,459,209,873.00","15,182,651,980.00",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,Wireless Telecommunication Services,2023Q1,VGI,"50,502,804,475,633.00","29,690,953,206,043.00","30,438,112,000,000.00","14,981,026,894,760.00","1,951,143,989,365.00","6,481,102,464,663.00","3,247,601,469,143.00","858,016,509,361.00","906,187,839,061.00","761,262,334,570.00","1,335,606,090,668.00","959,198,943,080.00","594,542,412,778.00","64,833,178,560,000.00","2,864,186,726,881.00","-1,154,996,431,978.00","-921,206,772,603.00",
858,Wireless Telecommunication Services,2023Q2,VGI,"48,424,867,370,960.00","28,750,124,791,181.00","30,438,112,000,000.00","12,511,301,384,221.00","2,453,567,719,032.00","6,861,419,133,040.00","3,138,944,970,744.00","-1,262,333,589,000.00","1,257,479,389,190.00","783,410,690,179.00","2,701,183,624,335.00","-811,793,476,997.00","-1,219,197,126,097.00","71,225,182,080,000.00","2,329,928,875,526.00","-940,627,565,132.00","-1,141,710,897,621.00",
859,Wireless Telecommunication Services,2023Q3,VGI,"50,803,353,975,670.00","29,914,708,205,697.00","30,438,112,000,000.00","11,943,237,946,185.00","2,827,815,035,891.00","7,325,551,939,115.00","3,749,958,955,757.00","1,983,950,432,695.00","859,055,376,619.00","1,004,279,680,321.00","1,518,104,994,957.00","2,098,074,484,066.00","1,409,588,119,101.00","70,312,038,720,000.00","5,230,127,845,525.00","-3,292,088,315,583.00","-1,115,463,343,252.00",
860,Wireless Telecommunication Services,2023Q4,VGI,"52,371,351,113,757.00","30,362,756,269,123.00","30,438,112,000,000.00","10,944,320,309,111.00","2,514,299,769,920.00","7,563,571,969